In [2]:
import numpy as np
from itertools import combinations
from scipy.linalg import orth
from scipy.optimize import linprog

def is_linearly_independent(vectors):
    return np.linalg.matrix_rank(vectors) == len(vectors)

def closure(matrix, subset):
    n = matrix.shape[1]
    current_set = set(subset)
    current_rank = np.linalg.matrix_rank(matrix[:, list(current_set)])
    for i in range(n):
        if i in current_set:
            continue
        new_set = current_set | {i}
        new_rank = np.linalg.matrix_rank(matrix[:, list(new_set)])
        if new_rank == current_rank:
            current_set.add(i)
    return current_set

def find_flats(matrix):
    n = matrix.shape[1]
    flats = set()
    for r in range(1,n + 1):
        for subset in combinations(range(n), r):
            cl = closure(matrix, subset)
            flats.add(tuple(sorted(cl)))
    return [set(flat) for flat in flats]

def generate_flags(flats):
    flats_sorted = sorted(flats, key=lambda x: (len(x), sorted(x)))
    flags = []
    def dfs(current_flag, remaining_flats):
        for i, flat in enumerate(remaining_flats):
            if not current_flag or all(f in flat for f in current_flag[-1]) and len(flat) > len(current_flag[-1]):
                new_flag = current_flag + [flat]
                flags.append(new_flag)
                dfs(new_flag, remaining_flats[i+1:])
    dfs([], flats_sorted)
    return flags

def project_to_quotient(flat, total_elements):
    vector = [0] * total_elements
    for element in flat:
        vector[element] = 1
    return vector

def fan_map(bergman_fan, V, tol=1e-8):
    cones = []
    for flag in bergman_fan:
        cone_generators = [project_to_quotient(F, V.shape[1]) for F in flag]
        mapped_cone = []
        for generator in cone_generators:
            vec = V.dot(np.array(generator))
            # Проверяем на нулевой вектор и NaN
            if np.linalg.norm(vec) > tol and not np.any(np.isnan(vec)):
                mapped_cone.append(vec)
        if mapped_cone:
            cones.append(mapped_cone)
    return cones

def find_extreme_rays(cones, tol=1e-8):
    extreme_rays = []
    for cone in cones:
        for vector in cone:
            # Проверяем на NaN и бесконечность
            if np.any(np.isnan(vector)) or np.any(np.isinf(vector)):
                continue
                
            # Нормализуем вектор
            norm = np.linalg.norm(vector)
            if norm < tol:  # Игнорируем нулевые векторы
                continue
                
            normalized = vector / norm
            
            # Проверяем на уникальность с учетом допуска
            is_unique = True
            for existing in extreme_rays:
                if np.allclose(normalized, existing, atol=tol):
                    is_unique = False
                    break
                    
            if is_unique:
                extreme_rays.append(normalized)
                
    return extreme_rays

# Example usage
matrix = np.array([
    [-4, 0, 0],
    [0, -4, 0],
    [0, 0, -4],
    [2, 2, 4],
    [2, 4, 2],
    [-2, -2, 4],
    [2, 4, -2]
]).T

V = np.array([
    [4, 0, 0, -2, -2, -2, 2],
    [0, 4, 0, -2, -4, -2, 4],
    [0, 0, 4, -4, -2, 4, -2]
])

flats = find_flats(matrix)
flags = generate_flags(flats)
bergman_fan = flags
mapped_cones = fan_map(bergman_fan, V)
extreme_rays = find_extreme_rays(mapped_cones)

print("Found extreme rays:", len(extreme_rays))
for ray in extreme_rays:
    if(abs(ray[0])>0):
        print(ray)

Found extreme rays: 22
[1. 0. 0.]
[0.70710678 0.70710678 0.        ]
[0.70710678 0.         0.70710678]
[ 0.40824829 -0.40824829 -0.81649658]
[ 0.40824829 -0.81649658 -0.40824829]
[ 0.40824829 -0.40824829  0.81649658]
[ 0.80178373  0.53452248 -0.26726124]
[-0.40824829  0.40824829 -0.81649658]
[-0.70710678  0.         -0.70710678]
[-0.40824829  0.40824829  0.81649658]
[ 0.23570226  0.94280904 -0.23570226]
[-0.57735027 -0.57735027  0.57735027]
[-0.40824829 -0.40824829 -0.81649658]
[-0.42640143 -0.63960215 -0.63960215]
[-0.40824829 -0.81649658 -0.40824829]
[-0.53452248 -0.80178373  0.26726124]
[-0.40824829 -0.40824829  0.81649658]
[ 0.40824829  0.81649658 -0.40824829]


In [3]:
from math import gcd
from functools import reduce
import numpy as np
from fractions import Fraction

def normalize_integer_vector(vec, tol=1e-8):
    """
    Приводит вектор к целочисленному виду с минимальными координатами.
    
    Параметры:
        vec: numpy array - входной вектор
        tol: float - допустимая погрешность
        
    Возвращает:
        numpy array - целочисленный вектор с минимальными координатами
    """
    # Проверяем на нулевой вектор
    if np.allclose(vec, 0, atol=tol):
        return np.zeros_like(vec, dtype=int)
    
    # Находим наименьший множитель для приведения к целым числам
    decimals = []
    for x in vec:
        # Находим десятичное представление с ограниченной точностью
        decimal = Fraction(x).limit_denominator(1000)
        decimals.append(decimal)
    
    # Находим общий знаменатель
    denoms = [d.denominator for d in decimals]
    common_denom = reduce(lambda a, b: a * b // gcd(a, b), denoms)
    
    # Умножаем на общий знаменатель и округляем
    int_vec = np.round([x * common_denom for x in vec]).astype(int)
    
    # Находим НОД всех координат
    common_divisor = reduce(gcd, map(abs, int_vec))
    
    # Делим на НОД (если не ноль)
    if common_divisor != 0:
        int_vec = int_vec // common_divisor
    
    # Нормализуем знак (делаем первую ненулевую координату положительной)
    for i in range(len(int_vec)):
        if int_vec[i] != 0:
            if int_vec[i] < 0:
                int_vec = -int_vec
            break
    
    return int_vec

def postprocess_vectors(vectors, tol=1e-8):
    """
    Приводит список векторов к целочисленному виду и удаляет дубликаты.
    
    Параметры:
        vectors: list of numpy arrays - список векторов
        tol: float - допустимая погрешность
        
    Возвращает:
        list of numpy arrays - уникальные целочисленные векторы
    """
    # Приводим к целочисленному виду
    int_vectors = [normalize_integer_vector(v, tol) for v in vectors]
    
    # Удаляем дубликаты
    unique_vectors = []
    for vec in int_vectors:
        is_unique = True
        for existing in unique_vectors:
            if np.array_equal(vec, existing):
                is_unique = False
                break
        if is_unique:
            unique_vectors.append(vec)
    
    return unique_vectors

# Пример использования в конце вашего кода:
# Обрабатываем найденные векторы
finded_vecs = extreme_rays.copy()
processed_vectors = postprocess_vectors(finded_vecs)

print("Целочисленные нормали:")
for vec in processed_vectors:
    print(vec)

Целочисленные нормали:
[1 0 0]
[1 1 0]
[1 0 1]
[ 1 -1 -2]
[ 1 -2 -1]
[ 1 -1  2]
[ 3  2 -1]
[0 1 0]
[0 1 1]
[ 1  4 -1]
[0 0 1]
[ 1  1 -1]
[1 1 2]
[128115 192172 192172]
[ 0  1 -3]
[1 2 1]
[ 2  3 -1]
[ 1  1 -2]
[ 1  2 -1]
